In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)


In [ ]:
df = pd.read_csv("../data/processed/sentiment_labeled_data.csv")
df["time"] = pd.to_datetime(df["time"])

print(df.shape)


In [ ]:
df["strategy_fear_accumulation"] = np.where(
    (df["Classification"] == "Fear") & (df["side_binary"] == 1),
    df["normalized_pnl"],
    0
)


In [ ]:
df["strategy_greed_distribution"] = np.where(
    (df["Classification"] == "Greed") & (df["side_binary"] == -1),
    df["normalized_pnl"],
    0
)


In [ ]:
df["strategy_risk_adaptive"] = np.where(
    ((df["Classification"] == "Fear") & (df["leverage"] <= df["leverage"].median())) |
    ((df["Classification"] == "Greed") & (df["leverage"] > df["leverage"].median())),
    df["normalized_pnl"],
    0
)


In [ ]:
df["strategy_contrarian"] = np.where(
    ((df["Classification"] == "Fear") & (df["side_binary"] == -1)) |
    ((df["Classification"] == "Greed") & (df["side_binary"] == 1)),
    df["normalized_pnl"],
    0
)


In [ ]:
strategies = [
    "strategy_fear_accumulation",
    "strategy_greed_distribution",
    "strategy_risk_adaptive",
    "strategy_contrarian"
]

results = {}

for strat in strategies:
    results[strat] = {
        "total_pnl": df[strat].sum(),
        "avg_pnl": df[strat].mean(),
        "win_rate": (df[strat] > 0).mean(),
        "trade_count": (df[strat] != 0).sum()
    }

strategy_results = pd.DataFrame(results).T
display(strategy_results)


In [ ]:
plt.figure(figsize=(12,6))

for strat in strategies:
    cumulative = df[strat].cumsum()
    plt.plot(cumulative, label=strat)

plt.legend()
plt.title("Cumulative PnL by Strategy")
plt.xlabel("Trades")
plt.ylabel("Cumulative Normalized PnL")
plt.show()


In [ ]:
strategy_results["profit_factor"] = (
    df[strategies].clip(lower=0).sum() /
    abs(df[strategies].clip(upper=0).sum())
).values

display(strategy_results)


In [ ]:
strategy_results.to_csv("../data/processed/strategy_performance.csv")
df[strategies].to_csv("../data/processed/strategy_trade_level.csv", index=False)

print("✅ Strategy simulation results saved.")
